In [7]:
pip install xlrd==2.0.1

  Using cached xlrd-2.0.1-py2.py3-none-any.whl (96 kB)
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.2.0
    Uninstalling xlrd-1.2.0:
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\programdata\\anaconda3\\lib\\site-packages\\xlrd-1.2.0.dist-info\\INSTALLER'
Consider using the `--user` option or check the permissions.



In [1]:
import glob
from glob import glob
import shutil
import requests
import urllib
import itertools
from requests.exceptions import ConnectionError, HTTPError, Timeout, TooManyRedirects
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import traceback
import logging
import logging.handlers
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import time as ti
import datetime
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import re
import os
import asyncio
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import ctypes
import threading
import os

global sheets
#fp = f'{os.path.basename(os.getcwd())}'
#filepath = fr'{fp}'
filepath = fr'{os.getcwd()}'
folderA = r'Archive'
filepathA = os.path.join(filepath, folderA)
folderB = r'xlsx'
filepathB = os.path.join(filepathA, folderB)
folderC = r'csv'
filepathC = os.path.join(filepathA, folderC)

try:
    
    logging.basicConfig(filename = 'RepliconCSVConversion.log',
                        filemode='a',
                        format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                        datefmt='%H:%M:%S',
                        level=logging.DEBUG)
    logging.info(f'\nLOG START: {datetime.now()}\n')
    
    def check_folders():
        if not os.path.exists(filepathA):
            os.makedirs(filepathA)
            print('Archives did not exist')
        if not os.path.exists(filepathB):
            os.makedirs(filepathB)
            print('xlsx did not exist')
        if not os.path.exists(filepathC):
            os.makedirs(filepathC)
            print('csv did not exist')
        else:
            print('Existing Archive Directories Detected')
        return

    def check_files():
        global processlist
        processlist = []
        for file in os.listdir(filepath):
            print(f'{file}')
            if "Da Vinci" in file and file.endswith((".xlsx",".xls",".xlsm")):
                print(f'\n{file}')
                processlist += [file]
            if "galaxy all" in file and file.endswith((".xlsx",".xls",".xlsm")):
                print(f'\n{file}')
                processlist += [file]
            if "SG & NZ" in file and file.endswith((".xlsx",".xls",".xlsm")):
                print(f'\n{file}')
                processlist += [file]
        if processlist == []:
            print('No Excel files were present')
            processlist += ['None']
        else:
            pass
        return processlist
    
    def convert_to_CSV(doc):
        all_sheets = pd.read_excel(f'{filepath}'+'/'+f'{doc}', sheet_name=None)
        sheets = all_sheets.keys()
        docdir = doc.replace('.xlsx','').replace('.xls','').replace('.xlsm','')
        filepathdoc = os.path.join(filepath, docdir)
        print(sheets, '\n\n')
        print(len(sheets), 'sheets' , '\n\n')
        print(list(sheets), '\n\n')
        print(doc)
        
        sheetlist = list(sheets)
        if len(sheets) > 1:
            if not os.path.exists(docdir):
                os.makedirs(docdir)
            for sheet_name in sheets:
                sheet = pd.read_excel(f'{filepath}'+'/'+f'{doc}', sheet_name=sheet_name).to_csv(f'{filepathdoc}'+'/'+f'{docdir}' + '_' + f'{sheet_name}' + '.csv', encoding = 'utf-8', index = False)
        else:
            pd.read_excel(f'{filepath}'+'/'+f'{doc}', sheet_name=sheetlist[0]).to_csv(f'{filepath}'+'/'+f'{docdir}' + '.csv', encoding = 'utf-8', index = False)
            #shutil.move(f'{filepath}'+'/'+f'{doc}', f'{filepathB}'+f'{doc}')
    
    def move_processed():
        global processedlist
        processedlist = []
        for file in os.listdir(filepath):
            print(f'{file}')
            if file.startswith("DDRProcessed") and file.endswith(".csv"):
                print(f'\n{file}')
                processedlist += [file]
            elif file.startswith(("Archive","Analysis","Temp")):
                pass
            elif ~file.startswith("DDRProcessed"):
                print('incorrect file name is present.')
        if processedlist == []:
            print('No files were present or did not have the correct naming scheme.')
            processedlist += ['None']
        else:
            pass
        return processedlist
    
    check_folders() 
    check_files()
    if 'None' not in processlist:
        print('\ndocuments available for processing\n')
        print(processlist)
        for doc in enumerate(processlist):
            try:
                gstart_time = datetime.now()
                convert_to_CSV(doc[1])
                gend_time = datetime.now()
                print("")
                print('\nDuration: {}'.format(gend_time - gstart_time))
                logging.info('\nDuration: {}'.format(gend_time - gstart_time))
            except Exception as e:
                print("")
                print(e)
                logging.exception('\n')
                logging.exception("message")
            for i in range(10, -1, -1):
                    print(f"{i} seconds until next document is processed", end = "\r")
                    ti.sleep(1)
    else:
        print('documents not available for processing')
        pass
    
    #if 'None' not in processedlist:
    #    for file in processedlist:
    #        shutil.move(f'{filepath}'+'/'+f'{processedlist[0]}', 'DataAndDuplicateReportAutomation/Archive/csv/'+f'{processedlist[0]}')
    #        move_processed()
    #    print('Processing Complete')
    #else:
    #    pass
    
#exceptions 
except (Exception, pyodbc.DatabaseError) as error:
        print(error)
        logging.exception("message")
        pass
    
except requests.exceptions.HTTPError as errh:
    print("Http Error:",  errh)
    conn = False
    logging.exception("message")
    
except requests.exceptions.ConnectionError as errc:
    print("Error Connecting:", errc)
    conn = False
    logging.exception("message")
    
except requests.exceptions.Timeout as errt:
    print("Timeout Error:", errt)
    conn = False
    logging.exception("message")
    
except requests.exceptions.RequestException as erru:
    print("Unidentified Request Exception:", erru)
    conn = False
    logging.exception("message")
        
finally:
    logging.info(f'\nLOG END: {datetime.now()}\n')

Existing Archive Directories Detected
.ipynb_checkpoints
Archive
RepliconCSVConversion.log
RepliconCSVPreprocessor-Copy1.ipynb
RepliconCSVPreprocessor.py
User List Report 2022-11-30 Da Vinci.xls

User List Report 2022-11-30 Da Vinci.xls
User List Report 2022-11-30 galaxy all.xls

User List Report 2022-11-30 galaxy all.xls
User List Report 2022-11-30 SG & NZ.xls

User List Report 2022-11-30 SG & NZ.xls

documents available for processing

['User List Report 2022-11-30 Da Vinci.xls', 'User List Report 2022-11-30 galaxy all.xls', 'User List Report 2022-11-30 SG & NZ.xls']
dict_keys(['Report']) 


1 sheets 


['Report'] 


User List Report 2022-11-30 Da Vinci.xls


Duration: 0:00:02.574813
dict_keys(['Report']) ocument is processedd


1 sheets 


['Report'] 


User List Report 2022-11-30 galaxy all.xls


Duration: 0:00:00.130284
dict_keys(['Report']) ocument is processedd


1 sheets 


['Report'] 


User List Report 2022-11-30 SG & NZ.xls


Duration: 0:00:00.921026
